In [1]:
# TEST THE CODE

import os
import requests
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from pdbfixer import PDBFixer
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Download the PDB file for 1UBQ
pdb_id = "1UBQ"
pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
pdb_filename = f"{pdb_id}.pdb"

if not os.path.exists(pdb_filename):
    response = requests.get(pdb_url)
    with open(pdb_filename, 'wb') as f:
        f.write(response.content)

# Step 2: Prepare the system using PDBFixer
fixer = PDBFixer(filename=pdb_filename)
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens()

# Define simulation parameters
temperature = 300 * unit.kelvin
timestep = 2.0 * unit.femtoseconds
simulation_time = 1 * unit.nanoseconds
n_steps = int(simulation_time / timestep)

# Step 3: Set up and run the simulation at low pressure (1 atm)
def run_simulation(fixer, pressure, output_dcd, output_log):
    forcefield = app.ForceField('amber99sb.xml', 'tip3p.xml')
    system = forcefield.createSystem(fixer.topology, nonbondedMethod=app.PME, 
                                     nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds)
    system.addForce(mm.MonteCarloBarostat(pressure, temperature))

    integrator = mm.LangevinIntegrator(temperature, 1.0/unit.picoseconds, timestep)
    simulation = app.Simulation(fixer.topology, system, integrator)
    simulation.context.setPositions(fixer.positions)

    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(temperature)

    simulation.reporters.append(app.DCDReporter(output_dcd, 1000))
    simulation.reporters.append(app.StateDataReporter(output_log, 1000, step=True, 
                                                      potentialEnergy=True, temperature=True, density=True))

    simulation.step(n_steps)

# Run low pressure simulation
run_simulation(fixer, 1 * unit.atmospheres, 'low_pressure.dcd', 'low_pressure.log')

# Step 4: Set up and run the simulation at high pressure (2000 atm)
run_simulation(fixer, 2000 * unit.atmospheres, 'high_pressure.dcd', 'high_pressure.log')

# Step 5: Analyze the RMSF for both simulations
def calculate_rmsf(dcd_file, pdb_file):
    traj = md.load(dcd_file, top=pdb_file)
    traj.superpose(traj, 0)
    rmsf = md.rmsf(traj, traj, 0)
    return rmsf

rmsf_low = calculate_rmsf('low_pressure.dcd', pdb_filename)
rmsf_high = calculate_rmsf('high_pressure.dcd', pdb_filename)

# Plot RMSF
plt.figure()
plt.plot(rmsf_low, label='Low Pressure (1 atm)')
plt.plot(rmsf_high, label='High Pressure (2000 atm)')
plt.xlabel('Residue')
plt.ylabel('RMSF (nm)')
plt.legend()
plt.title('RMSF Comparison')
plt.show()

# Step 6: Calculate and plot the moments of inertia over time for both simulations
def calculate_moments_of_inertia(traj):
    moments = []
    for frame in traj:
        inertia_tensor = md.geometry.compute_inertia_tensor(frame)
        eigenvalues = np.linalg.eigvals(inertia_tensor)
        moments.append(eigenvalues)
    return np.array(moments)

traj_low = md.load('low_pressure.dcd', top=pdb_filename)
traj_high = md.load('high_pressure.dcd', top=pdb_filename)

moments_low = calculate_moments_of_inertia(traj_low)
moments_high = calculate_moments_of_inertia(traj_high)

# Plot moments of inertia
plt.figure()
plt.plot(moments_low[:, 0], label='Low Pressure I1')
plt.plot(moments_low[:, 1], label='Low Pressure I2')
plt.plot(moments_low[:, 2], label='Low Pressure I3')
plt.plot(moments_high[:, 0], label='High Pressure I1', linestyle='--')
plt.plot(moments_high[:, 1], label='High Pressure I2', linestyle='--')
plt.plot(moments_high[:, 2], label='High Pressure I3', linestyle='--')
plt.xlabel('Frame')
plt.ylabel('Moment of Inertia (amu*nm^2)')
plt.legend()
plt.title('Moments of Inertia Over Time')
plt.show()

OpenMMException: The periodic box size has decreased to less than twice the nonbonded cutoff.

In [1]:
!ls

1UBQ.pdb      exp_13_test_code.ipynb  high_pressure.log  low_pressure.log
exp_13.ipynb  high_pressure.dcd       low_pressure.dcd
